# Step 2: Parse Document

Parse documents using Databricks AI Document Intelligence.

**Pipeline:** autoloader  
**Step:** parse_document  
**Service:** document_parser_default

**Inputs:**
- Source: Bronze table from `read_documents` step
- Format: PDF documents (binary content)

**Outputs:**
- Table: `${bronze_schema}.autoloader_documents_parsed`
- Extracted text, tables, and metadata


In [ ]:
# Imports
from pyspark.sql import functions as F
from pyspark.sql.types import *
import json

# Load configuration from task variables (set by setup task)
config_json = dbutils.jobs.taskValues.get(taskKey="setup", key="config")
config = json.loads(config_json)

print("✅ Configuration loaded from task variables")


In [ ]:
# Get step configuration
pipeline_config = config["pipelines"]["autoloader"]
step_config = pipeline_config["pdf"]["standard"]["steps"][1]  # parse_document step (index 1, after read_documents)

service_name = step_config["service"]
service_config = config["services"][service_name]

source_table = step_config["source"]  # Input from previous step (read_documents)
output_table = step_config["output"]

print(f"Service: {service_name}")
print(f"Source Table: {source_table}")
print(f"Output Table: {output_table}")


In [ ]:
# Read from Bronze table (output of read_documents step)
df_documents = spark.table(source_table)

print(f"Documents to process: {df_documents.count()}")
df_documents.printSchema()


In [ ]:
# Parse PDF documents using AI Document Intelligence
# TODO: Replace with actual Databricks AI Document Intelligence API call

# For now, creating placeholder extracted_text column
df_parsed = (
    df_documents
    .withColumn("processing_timestamp", F.current_timestamp())
    .withColumn("processing_date", F.current_date())
    .withColumn("document_id", F.md5(F.col("file_path")))
    # TODO: Call AI Document Intelligence service here
    .withColumn("extracted_text", F.lit("Placeholder for extracted text - use AI Document Intelligence"))
    .withColumn("extracted_tables", F.array())
    .withColumn("metadata", F.struct())
    .select(
        "document_id",
        "file_name",
        "file_path",
        "file_content",
        "extracted_text",
        "extracted_tables",
        "metadata",
        "processing_date",
        "processing_timestamp"
    )
)

print(f"✅ Processed {df_parsed.count()} documents")


In [ ]:
# Write to output table
(
    df_parsed.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(output_table)
)

print(f"✅ Data written to {output_table}")
